In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.metrics import precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import KFold
np.random.seed(2018)

In [3]:
from sklearn.model_selection import KFold, StratifiedKFold

In [4]:
df2016 = pd.read_csv('../output/GPS2016c.csv')
df2017 = pd.read_csv('../output/GPS2017c.csv')
df2018 = pd.read_csv('../output/GPS2018c.csv')

In [5]:
df2016['rok'] = 2016
df2017['rok'] = 2017
df2018['rok'] = 2018
df = pd.concat([df2016, df2017, df2018])

In [6]:
df.columns = ['GPS_X_stopnie', 'GPS_X_minuty', 'GPS_X_sekundy', 'GPS_Y_stopnie', 'GPS_Y_minuty','GPS_Y_sekundy',
             'Zdarzenia', 'Wypadki', 'Zabici', 'Ranni', 'miejscowość', 'droga', 'ulica','rok']

lista = df.groupby('miejscowość')['Wypadki'].agg( np.mean)
def mean_woj (value):
    return (lista[value])

df['miejscowość_wypadek'] = df['miejscowość'].map(mean_woj)

lista = df.groupby('droga')['Wypadki'].agg( np.mean)
def mean_woj (value):
    return (lista[value])

df['droga_wypadek'] = df['droga'].map(mean_woj)

lista = df.groupby('ulica')['Wypadki'].agg( np.mean)
def mean_woj (value):
    return (lista[value])

df['ulica_wypadek'] = df['ulica'].map(mean_woj)

In [7]:
df2016 = df[df['rok']==2016]
df2017 = df[df['rok']==2017]
df2018 = df[df['rok']==2018]
df2016.drop(columns ='rok', inplace=True)
df2017.drop(columns ='rok', inplace=True)
df2018.drop(columns ='rok', inplace=True)



/Users/kris/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
X_train = df2016.values
y_train = df2017.iloc[:,7].map (lambda x : 1 if x>0 else 0)
X_test = df2017.values
y_test = df2018.iloc[:,7].map (lambda x : 1 if x>0 else 0)

In [9]:
print('Metryka sukcesu dla metody naiwnej',precision_score(y_test, y_train))
print (y_train.sum())
print (y_test.sum())

Metryka sukcesu dla metody naiwnej 0.18242343541944075
3004
2741


In [12]:
model1 = xgb.XGBClassifier(max_depth=3, n_estimators=10, seed=2018, learning_rate=0.1)
model1.fit(X_train, y_train)
y_predXGB = model1.predict(X_test)
print(precision_score(y_test, y_predXGB))
print ("Ilość wytypowanych obszarów: ",y_predXGB.sum())
print ("Ilość rzeczywistych oszarów: ",y_test.sum())

0.6883116883116883
Ilość wytypowanych obszarów:  77
Ilość rzeczywistych oszarów:  2741


In [10]:
model = xgb.XGBClassifier(max_depth=4, n_estimators=30, seed=2018)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(precision_score(y_test, y_pred))
print ("Ilość wytypowanych obszarów: ",y_pred.sum())
print ("Ilość rzeczywistych oszarów: ",y_test.sum())

0.5670498084291188
Ilość wytypowanych obszarów:  261
Ilość rzeczywistych oszarów:  2741


In [13]:
y_pred = pd.Series(y_pred)
y_test = pd.Series(y_test)

In [14]:
lista = y_pred[y_pred==1].index
lista.shape

(261,)

In [15]:
dfGPS= pd.DataFrame(np.random.randint(0,1, (lista.shape[0], 11)))

In [16]:
for i in range (lista.shape[0]):
    dfGPS.iloc[i,0] = df2018.iloc[lista[i],0]
    dfGPS.iloc[i,1] = df2018.iloc[lista[i],1]
    dfGPS.iloc[i,2] = df2018.iloc[lista[i],2]
    dfGPS.iloc[i,3] = df2018.iloc[lista[i],3]
    dfGPS.iloc[i,4] = df2018.iloc[lista[i],4]
    dfGPS.iloc[i,5] = df2018.iloc[lista[i],5]
    dfGPS.iloc[i,8] = df2018.iloc[lista[i],13]
    dfGPS.iloc[i,9] = df2018.iloc[lista[i],14]
    dfGPS.iloc[i,10] = df2018.iloc[lista[i],15]

In [17]:
dfGPS.iloc[:,2] = dfGPS.iloc[:,2].map(lambda x : x*10)
dfGPS.iloc[:,5] = dfGPS.iloc[:,5].map(lambda x : x*5)

In [18]:
dfGPS.shape

(261, 11)

In [20]:
dfGPS.iloc[:,7] = dfGPS.iloc[:,0].map(lambda x: x) + (dfGPS.iloc[:,1].map(lambda x: x)/60) + ((dfGPS.iloc[:,2].map(lambda x: x)+5)/3600)
dfGPS.iloc[:,6] = dfGPS.iloc[:,3].map(lambda x: x) + (dfGPS.iloc[:,4].map(lambda x: x)/60) + ((dfGPS.iloc[:,5].map(lambda x: x)+2.5)/3600)

In [21]:
dfGPS.head()

,0,1,2,3,4,5,6,7,8,9,10
0,18,33,20,49,56,30,49.942361,18.556944,0.215475,0.402597,0.416667
1,18,35,40,49,57,10,49.953472,18.595833,0.215475,0.388889,0.714286
2,18,36,50,49,56,45,49.946528,18.615278,0.215475,0.282609,0.485714
3,18,38,10,49,45,5,49.752083,18.637500,0.153710,0.086058,0.176471
4,18,38,20,49,44,50,49.747917,18.640278,0.153710,1.000000,1.000000


In [174]:
dfGPS.to_csv('../output/GPSWypadki2018(56%).csv', index=False, header=True)

# Predykcja 2019

In [10]:
X_test_2018 = df2018.values

In [11]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test_2018)
print (y_pred.sum())

239


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
y_pred = pd.Series(y_pred)
lista = y_pred[y_pred==1].index
lista.shape

(239,)

In [193]:
dfGPS= pd.DataFrame(np.random.randint(0,1, (lista.shape[0], 10)))

In [194]:
for i in range (lista.shape[0]):
    dfGPS.iloc[i,0] = df2018.iloc[lista[i],0]
    dfGPS.iloc[i,1] = df2018.iloc[lista[i],1]
    dfGPS.iloc[i,2] = df2018.iloc[lista[i],2]
    dfGPS.iloc[i,3] = df2018.iloc[lista[i],3]
    dfGPS.iloc[i,4] = df2018.iloc[lista[i],4]
    dfGPS.iloc[i,5] = df2018.iloc[lista[i],5]

In [195]:
dfGPS.iloc[:,2] = dfGPS.iloc[:,2].map(lambda x : x*10)
dfGPS.iloc[:,5] = dfGPS.iloc[:,5].map(lambda x : x*5)

In [196]:
dfGPS.iloc[:,6] = dfGPS.iloc[:,0].map(lambda x: x) + (dfGPS.iloc[:,1].map(lambda x: x)/60) + ((dfGPS.iloc[:,2].map(lambda x: x)+5)/3600)
dfGPS.iloc[:,7] = dfGPS.iloc[:,3].map(lambda x: x) + (dfGPS.iloc[:,4].map(lambda x: x)/60) + ((dfGPS.iloc[:,5].map(lambda x: x)+2.5)/3600)

In [197]:
dfGPS.to_csv('../output/PredykcjaWypadki2019(56%).csv', index=False, header=True)

In [14]:
df2019 = pd.read_csv('../output/GPS2019.csv')

In [15]:
y_test = df2019.iloc[:,6].map (lambda x : 1 if x>0 else 0)

In [16]:
y_test.sum()

544

In [17]:
df2019.iloc[:,6].sum()

569

In [19]:
print('predykcja wypadków styczeń - luty',precision_score(y_test, y_pred))

predykcja wypadków styczeń - luty 0.11297071129707113


In [18]:
print('predykcja wypadków styczeń - marzec',precision_score(y_test, y_pred))

predykcja wypadków styczeń - marzec 0.19246861924686193
